In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

In [18]:
from datetime import date

def pretvori_datum_v_datetime(string_datuma):
    """
    Funkcija, ki obe vrsti datuma pretvori v ustretno datetime obliko (datum_izida in datum all_time_high/low)
    """
    
    # Za vsak slučaj
    if not string_datuma:
        return None
    
    # Ločimo primera 'Apr 29, 2022' in '2022-04-29'
    if "," in string_datuma:
        return datetime.strptime(string_datuma, "%b %d, %Y")
    else:
        return datetime.strptime(string_datuma, "%Y-%m-%d")
    
print(pretvori_datum_v_datetime("Apr 9, 2022"))
print(pretvori_datum_v_datetime("2022-04-09"))

2022-04-09 00:00:00
2022-04-09 00:00:00


In [23]:
def popravi_cardtype_aftermath(seznam_cardtypa):
    """
    Popravimo seznam cardtypa, ki se pojavi zaradi nenavadnega/nekonsistentnega zapisa Aftermath kart na spletni strani
    """
    
    popravljen_seznam = []
    for tip in seznam_cardtypa:
        if tip not in popravljen_seznam and tip != "//":
            popravljen_seznam.append(tip)
    return popravljen_seznam

print(popravi_cardtype_aftermath(["Sorcery", "//", "Sorcery"]))

['Sorcery']


In [31]:
def popravi_subtype_adventure(seznam_subtypa):
    """
    Popravimo seznam subtypa, ki se pojavi zaradi nenavadnega/nekonsistentnega zapisa Adventure kart na spletni strani
    """
    
    popravljen_seznam = []
    for tip in seznam_subtypa:
        if tip == "//":
                break
        elif tip not in popravljen_seznam:
            popravljen_seznam.append(tip)
    return popravljen_seznam

print(popravi_subtype_adventure(['Beast', 'Noble', '//', 'Sorcery']))

['Beast', 'Noble']


In [32]:
problematicni_tipi = ["Token", "Emblem"]
def vsebovanost_problematicnih_karte_na_podlagi_seznama(seznam, seznam_problematicnih_elementov):
    """
    Preverimo, ali določen seznam vsebuje problematične elemente
    """
    
    vsebuje_problem = False
    for element in seznam:
        if element in seznam_problematicnih_elementov:
            vsebuje_problem = True
            break
    return vsebuje_problem

print(vsebovanost_problematicnih_karte_na_podlagi_seznama(["Nissa"], problematicni_tipi))

False


In [41]:
dodatne_lastnosti_kart = pd.read_csv("specificne_lastnosti_kart.csv")
pojavitve_barve = dodatne_lastnosti_kart.groupby("barva").size()
verjetnosti_barve = pojavitve_barve / len(dodatne_lastnosti_kart)

verjetnosti_barve

barva
b        0.1465
bg       0.0120
bgu      0.0010
br       0.0145
brg      0.0005
c        0.1505
g        0.1390
gu       0.0165
gur      0.0010
gw       0.0150
gwu      0.0005
r        0.1445
rg       0.0150
rgw      0.0005
ru       0.0010
rw       0.0095
rwb      0.0010
u        0.1330
ub       0.0095
ubr      0.0015
ug       0.0010
ur       0.0100
urw      0.0005
uw       0.0005
w        0.1445
wb       0.0135
wbg      0.0005
wu       0.0145
wub      0.0005
wubrg    0.0020
dtype: float64

In [53]:
def odstrani_konce(beseda, konci):
    for konec in konci:
        if beseda.endswith(konec):
            return beseda[:-len(konec)]
    return beseda
    
def koren_besede(beseda):
    beseda = ''.join(znak for znak in beseda if znak.isalpha())
    if not beseda:
        return '$'
    beseda = odstrani_konce(beseda, ["ing", "ed", "en", "s"])
    beseda = beseda.rstrip('aeiou')
    if not beseda:
        return '@'
    return beseda

def koreni_besed(niz):
    return pd.Series(sorted({
        koren_besede(beseda) for beseda in str(niz).replace('-', ' ').lower().split() if beseda
    }))

In [52]:
opis = """Whenever another creature enters the battlefield under your control, you gain life equal to that creature's toughness.
(1)(g)(w), (tap): Populate. (Put a token onto the battlefield that's a copy of a creature token you control.)"""
koreni_besed(opis)

0               @
1         another
2     battlefield
3         control
4            copy
5         creatur
6           enter
7           equal
8            gain
9              gw
10            lif
11             of
12            ont
13        populat
14            put
15              t
16            tap
17             th
18           that
19            tok
20       toughnes
21          under
22       whenever
23              y
24           your
dtype: object

In [98]:
razpredelnica_s_koreni_oracle_textov = dodatne_lastnosti_kart.oracle_text.apply(koreni_besed)
print(razpredelnica_s_koreni_oracle_textov)

           0            1        2        3          4         5           6   \
0           @   controller  counter     exil  graveyard        if     instead   
1           @     addition      and   beacon       bolt        by        card   
2           $          and       at   attack  battalion       bor     creatur   
3           @  battlefield     card    enter       from  glowspor   graveyard   
4           $           an     cast    damag       deal      each  guttersnip   
...       ...          ...      ...      ...        ...       ...         ...   
1995        $            @  creatur        d       gain        if         lif   
1996        @           at   beginn      fly       forc        of         pay   
1997        $       attack     cant  creatur   defender       end         fir   
1998  creatur         hand       it    owner     return         t      target   
1999  creatur       doesnt   effect  enchant       from         h  protection   

              7         8  

In [103]:
koreni_tekstov = (
    razpredelnica_s_koreni_oracle_textov.unstack(  # vnose v razpredelnici razpotegnemo v eno vrsto (z gnezdenim indeksom)
    ).reset_index(                          # podatke iz indeksa pretvorimo v razpredelnico
    )[
        ["level_1", 0]                           # izberemo le relevantna dva stolpca
    ].rename(
        {"level_1": "indeks_oracle_teksta", 0: "koren"},
        axis = 1    # stolpcema damo smiselni imeni                            # s tem povemo, da preimenujemo stolpce in ne vrstic
    ).dropna(                               # odstranimo manjkajoče vnose
    )
)
koreni_tekstov

,indeks_oracle_teksta,koren
0,0,@
1,1,@
2,2,$
3,3,@
4,4,$
...,...,...
113015,1015,tw
115015,1015,up
117015,1015,with
119015,1015,y


In [131]:
#
tabela_barv = dodatne_lastnosti_kart[["barva"]]
tabela_barv.index.name = "indeks_oracle_teksta"
#print(koreni_tekstov, tabela_barv)

skupna_tabela = pd.merge(koreni_tekstov, tabela_barv, left_on="indeks_oracle_teksta", right_on="indeks_oracle_teksta")
print(skupna_tabela)

       indeks_oracle_teksta       koren barva
0                         0           @     u
1                         0  controller     u
2                         0     counter     u
3                         0        exil     u
4                         0   graveyard     u
...                     ...         ...   ...
39004                  1999  protection     w
39005                  1999       remov     w
39006                  1999          th     w
39007                  1999        ward     w
39008                  1999        whit     w

[39009 rows x 3 columns]


In [160]:
stolpec_verjetnosti_korenov_pri_oracle_tekstu = skupna_tabela.groupby(["barva", "koren"]).size() / pojavitve_barve
verjetnosti_korenov_pri_oracle_tekstu = stolpec_verjetnosti_korenov_pri_oracle_tekstu.unstack().transpose().fillna(1 / len(dodatne_lastnosti_kart))
#verjetnosti_korenov_pri_oracle_tekstu
print(verjetnosti_korenov_pri_oracle_tekstu)

barva            b        bg     bgu        br     brg         c         g  \
koren                                                                        
$         0.580205  0.666667  0.0005  0.620690  1.0000  0.568106  0.600719   
@         0.593857  0.625000  1.0000  0.482759  1.0000  0.564784  0.654676   
abilit    0.003413  0.000500  0.0005  0.000500  0.0005  0.013289  0.025180   
ability   0.020478  0.000500  1.0000  0.000500  0.0005  0.043189  0.028777   
abl       0.003413  0.000500  0.0005  0.000500  0.0005  0.006645  0.017986   
...            ...       ...     ...       ...     ...       ...       ...   
zenith    0.000500  0.000500  0.0005  0.000500  0.0005  0.000500  0.000500   
zhur      0.000500  0.000500  0.0005  0.000500  0.0005  0.000500  0.000500   
zomb      0.109215  0.000500  0.0005  0.034483  0.0005  0.006645  0.000500   
zon       0.000500  0.000500  0.0005  0.000500  0.0005  0.006645  0.003597   
zulaport  0.003413  0.000500  0.0005  0.000500  0.0005  0.000500

In [190]:
def ugani_barvo(opis):
    koreni_oracle_teksta = koreni_besed(opis)
    r = verjetnosti_korenov_pri_oracle_tekstu.reset_index()
    verjetnosti = r[r.koren.isin(koreni_oracle_teksta)].product(numeric_only=True) * verjetnosti_barve
    return verjetnosti.sort_values(ascending=False).head(10)

ugani_barvo("""
Put X 1/2 green Spider creature tokens with reach onto the battlefield, where X is the number of creatures attacking you. Prevent all combat damage that would be dealt this turn by non-Spider creatures
"""
)

barva
w      0.006500
b      0.004500
wb     0.001000
bg     0.000500
wub    0.000500
wu     0.000500
ub     0.000500
rw     0.000500
bgu    0.000500
c      0.000075
dtype: float64